In [ ]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
import netgen.gui

import matplotlib.pyplot as plt

# Example 10.2

Let $\Omega_1 = (0, 1) \times(0, 1)$ and $\Omega_2 =(1,2)\times(0,1)$. Further let $\Gamma_{int} :={1}\times(0,1)$ and $\Gamma_{out} :=(\partial \Omega_1\ u\ \partial \Omega_2)/ \Gamma_{int}$. Solve the problem

\begin{align}
-\Delta u_1 &= 10 &on\ \Omega_1 \\
-\Delta u_2 &= 0 &on\ \Omega_2 \\
u_1 = u_2 &= 0 &on\ \Gamma_{out} \\
u_1 - u_2 &= \epsilon \frac{\partial u_1}{\partial n_1} &on\ \Gamma_{int} \\
\frac{\partial u_1}{\partial n_1} &= -\frac{\partial u_2}{\partial n_2}
\end{align}

with $\epsilon = 0.1$.
We approximate both solutions in H1-conforming finite element spaces defined on Ω1 and Ω2 and use a mixed method to incorporate the boundary conditions. Use the file jump.py as starting point.

In [ ]:
# Mesh
geo = SplineGeometry()
p1,p2,p3,p4 = [ geo.AppendPoint(x,y) for x,y in [(0,0), (1,0), (1,1), (0,1)] ]
p5,p6 =  [ geo.AppendPoint(x,y) for x,y in [(2,0), (2,1)] ]

geo.Append (["line", p1, p2], leftdomain=1, rightdomain=0, bc='out')
geo.Append (["line", p2, p3], leftdomain=1, rightdomain=2, bc='int')
geo.Append (["line", p3, p4], leftdomain=1, rightdomain=0, bc='out')
geo.Append (["line", p4, p1], leftdomain=1, rightdomain=0, bc='out')
geo.Append (["line", p2, p5], leftdomain=2, rightdomain=0, bc='out')
geo.Append (["line", p5, p6], leftdomain=2, rightdomain=0, bc='out')
geo.Append (["line", p6, p3], leftdomain=2, rightdomain=0, bc='out')

geo.SetMaterial(1, 'left')
geo.SetMaterial(2, 'right')

mesh = Mesh(geo.GenerateMesh(maxh=0.1))
Draw(mesh)


print(mesh.GetMaterials())
print(mesh.GetBoundaries())

In [ ]:
### SETUP parameters for the problem as given in the exercise sheet ###
#######################################################################
dirichlet = 'out'
robin = 'int'
rhs = Parameter(0.0)
# Weak Dirichlet boundary condition
ud = Parameter(0.0) # CoefficientFunction
rhs1 = Parameter(10.0)
rhs2 = Parameter(0.0)
eps = Parameter(0.1)

# FES(es)
order = 3
Vl = H1(mesh, order = order, definedon = "left")
Vr = H1(mesh, order = order, definedon = "right")
Q = SurfaceL2(mesh, order = order-1)

fes = FESpace([Vl,Vr,Q])
(u1, u2, p), (v1, v2, q) = fes.TnT()
#print((u1, u2, p), (v1, v2, q))

# Bilinear Form
a = BilinearForm(fes) #,symmetric=True)
# Omega_1 == left
a += grad(u1) * grad(v1) *dx(definedon=mesh.Materials('left')) #
a += p * v1 * ds(definedon=mesh.Boundaries('out|int'))
# Interface
a += u1 * q * ds('out|int')
a += -u2 * q * ds('out|int')
a += eps * p * q * ds('int') #
# Omega_2 == right
a += grad(u2) * grad(v2) * dx('right') #
a += -p * v2 * ds('out|int')


# Setup solution GridFunction
# ud = 0 if bnd != left
gf = GridFunction(fes) # solution
gful = gf.components[0]
gfur = gf.components[1]
gfp = gf.components[2]

#gful.Set(ud, definedon=mesh.Boundaries('int')) # set boundary condition
#gfur.Set(ud, definedon=mesh.Boundaries('int')) # set boundary condition
#Draw(gful, mesh, "ul_BND")
#Draw(gfur, mesh, "ur_BND")

# Linear Form
f = LinearForm(fes)
f += rhs1 * v1 * dx('left') 
#f += -eps * q * ds('int')
f += rhs2 * v2 * dx('right') # probably don't need that cause rhs2=0 

In [ ]:
eps.Set(0.1)
with TaskManager():
    a.Assemble()
    f.Assemble()
    # Homogenize
    #r = f.vec.CreateVector()
    #r.data = f.vec - a.mat * gf.vec
    # Solve
    #gf.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * r
    solvers.BVP(bf=a, lf=f, gf=gf)


Draw (gful, mesh, "ul")
Draw (gfur, mesh, "ur")
Draw (gfp, mesh, "p")


#to draw the global solution
V = L2(mesh, order = order)
gfu_total = GridFunction(V)
gfu_total.Set(gful + gfur)
Draw(gfu_total, mesh, "u_total")
Draw(-grad(gfu_total), mesh, "flux_total")